In [5]:
from dataReadFromDisk import *

In [6]:
df = load_df_from_csv()

DataFrame loaded from: E:\\Notebooks\\osu\processed-metadata\processed-metadata.csv


In [7]:
df = df[:50]

In [8]:
dataset,num_of_batches = process_audio_and_beatmap_for_model_all_and_save(df)

In [9]:
num_of_batches

50

In [10]:
for input_data, output_data in dataset.take(1):
    print("First input:")
    print(input_data.numpy()[0][0])
    print("First output:")
    print(output_data.numpy()[0][0])

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

In [7]:
import numpy as np
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 155)),  # Input layer for variable length sequences with 155 features
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256 , return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256 , return_sequences=True)),
    # tf.keras.layers.Dense(64, activation='relu'),  # Optional dense layer
    tf.keras.layers.Dense(35)  # Output layer, size 35 to match your output data dimension
])

model.compile(optimizer='adam', loss=tf.keras.losses.Huber())

# Training the model
model.fit(dataset, epochs=10, steps_per_epoch=num_of_batches) # instead of 50 make it number of songs in final processed data

Epoch 1/10
13/50 ━━━━━━━━━━━━━━━━━━━━ 5:19 9s/step - loss: 29.9770

In [1]:
import tensorflow as tf

# Check if GPU is available
gpus = tf.config.experimental.list_physical_devices()
gpus

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.__version__

'2.17.0'

In [12]:
# Ensure that GPU is being used
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

# Model definition
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 155)),  # Input layer for variable length sequences with 155 features
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Dense(35)  # Output layer, size 35 to match your output data dimension
])

# Compile the model
model.compile(optimizer='adam', loss=tf.keras.losses.Huber())

# Training the model
model.fit(dataset, epochs=10, steps_per_epoch=num_of_batches)

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


NotImplementedError: Cannot convert a symbolic Tensor (bidirectional/forward_lstm_1/strided_slice:0) to a numpy array.